# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_csv = "output_data/cities_data_update.csv"

In [3]:
city_data = pd.read_csv(city_csv)
city_data

,Unnamed: 0,City,Country,Latitude,Longitude,Max Temperature(F),Humidity,Cloudiness,Wind Speed(mph),Date
0,0,mataura,NZ,-46.19,168.86,55.00,76.0,71.0,13.00,1.578369e+09
1,1,yellowknife,CA,62.45,-114.38,-25.01,74.0,75.0,4.70,1.578369e+09
2,2,kunashak,RU,55.70,61.55,23.00,85.0,75.0,6.71,1.578369e+09
3,3,geraldton,CA,49.72,-86.95,5.00,77.0,5.0,4.70,1.578369e+09
4,4,waddan,LY,29.16,16.14,43.32,74.0,17.0,6.78,1.578369e+09
...,...,...,...,...,...,...,...,...,...,...
637,637,barcelos,BR,-0.97,-62.93,77.58,76.0,21.0,1.72,1.578369e+09
638,638,sao joao da barra,BR,-21.64,-41.05,77.63,90.0,100.0,11.14,1.578369e+09
639,639,north bend,US,43.41,-124.22,52.00,100.0,90.0,10.29,1.578369e+09
640,640,sumbawa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
gmaps.configure(api_key=g_key)

In [5]:
city_data = city_data.dropna()

In [6]:
locations = city_data[["Latitude", "Longitude"]].astype(float)

In [7]:
humidity = city_data["Humidity"].astype(float)

In [8]:
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=90,
                                 point_radius = 2)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [9]:
#Already done above
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=90,
                                 point_radius = 2)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [20]:
hotel_df =  pd.DataFrame(city_data, columns=["City", "Country", "Latitude", "Longitude"])
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Latitude,Longitude,Hotel Name
0,mataura,NZ,-46.19,168.86,
1,yellowknife,CA,62.45,-114.38,
2,kunashak,RU,55.70,61.55,
3,geraldton,CA,49.72,-86.95,
4,waddan,LY,29.16,16.14,
...,...,...,...,...,...
636,sechura,PE,-5.56,-80.82,
637,barcelos,BR,-0.97,-62.93,
638,sao joao da barra,BR,-21.64,-41.05,
639,north bend,US,43.41,-124.22,


In [21]:
# using airport-rating class activity as example

# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "hotel",
    "key": g_key
}

# Use the lat/lng we recovered to identify hotels
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Latitude"]
    lng = row["Longitude"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    

    # convert to json
    name_address = name_address.json()

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/

In [22]:
hotel_df

,City,Country,Latitude,Longitude,Hotel Name
0,mataura,NZ,-46.19,168.86,Ellie's Villa
1,yellowknife,CA,62.45,-114.38,Chateau Nova Yellowknife
2,kunashak,RU,55.70,61.55,Gostinitsa Kunashak
3,geraldton,CA,49.72,-86.95,Victoria Bed & Breakfast
4,waddan,LY,29.16,16.14,
...,...,...,...,...,...
636,sechura,PE,-5.56,-80.82,Hotel Loma Dorada
637,barcelos,BR,-0.97,-62.93,Hotel Amazonita
638,sao joao da barra,BR,-21.64,-41.05,Pousada Kactus
639,north bend,US,43.41,-124.22,The Mill Casino • Hotel & RV Park


In [26]:
narrowed_city_df = hotel_df.dropna()

In [28]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [31]:
# Add marker layer ontop of heat map

fig = gmaps.figure()

markers = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(markers)
fig

# Display Map

Figure(layout=FigureLayout(height='420px'))